In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [27]:
def get_data(start):
    url='https://www.imdb.com/search/title/?release_date=2020-01-01,2020-12-31&sort=num_votes,desc&start='+str(start)+'&ref_=adv_nxt'
    r=requests.get(url)
    soup=BeautifulSoup(r.content,'html.parser')
    all=[]
    for d in soup.find_all('div',attrs={'class':'lister-item mode-advanced'}):
        all1=[]
        
        div_img=d.find('div',attrs={'class':'lister-item-image float-left'})
        movie_name=div_img.find('img',alt=True)
        
        imdb_rating_div=d.find('div',attrs={'class':'inline-block ratings-imdb-rating'})
        imdb_rating=imdb_rating_div.find('strong')
        
        description=d.find_all('p',attrs={'class':'text-muted'})[1]
        
        votes_p=d.find('p',attrs={'class':'sort-num_votes-visible'})
        votes=votes_p.find_all('span',attrs={'name':'nv'})[0]
        
        metascore=d.find('span',attrs={'class':'metascore'})
        
        x=d.find('p',attrs={'class':''})
        directors='NaN'
        stars='NaN'
        if x is not None:
            if 'Director' in x.text and 'Star' in x.text:
                s1 = x.text.find(':')
                s2 = x.text.find(':',s1+1)
                s3 = x.text.find('|')
                directors = x.text[s1+1:s3]
                stars = x.text[s2+1:]
            elif 'Director' in x.text:
                s1=x.text.find(':')
                directors=x.text[s1+1:]
            elif 'Star' in x.text:
                s1 = x.text.find(':')
                stars = x.text[s1+1:]
        
        genre_div=d.find_all('p',attrs={'class':'text-muted'})[0]
        genre=genre_div.find('span',attrs={'class':'genre'})
        
        runtime=genre_div.find('span',attrs={'class':'runtime'})
        
        if movie_name is not None:
            all1.append(movie_name.get('alt'))
        else:
            all1.append('NaN')
        if imdb_rating is not None:
            all1.append(imdb_rating.text)
        else:
            all1.append('NaN')
        if description is not None:
            all1.append(description.text[1:-1])
        else:
            all1.append('NaN')
        if votes is not None:
            all1.append(votes.text)
        else:
            all1.append('NaN')
        all1.append(directors.replace('\n', ''))
        all1.append(stars.replace('\n', ''))
        if metascore is not None:
            all1.append(metascore.text)
        else:
            all1.append('NaN')
        if genre is not None:
            all1.append(genre.text[1:])
        else:
            all1.append('NaN')
        if runtime is not None:
            all1.append(runtime.text)
        else:
            all1.append('NaN')
        all.append(all1)
    return all

In [28]:
start=1
result=[]
while start<=951:
    result.append(get_data(start))
    start+=50
flatten=lambda l: [item for sublist in l for item in sublist]
df=pd.DataFrame(flatten(result),columns=['Movie Name','Imdb Rating','Description','Votes','Directors','Stars','Metascore','Genre','Runtime'])
df

,Movie Name,Imdb Rating,Description,Votes,Directors,Stars,Metascore,Genre,Runtime
0,Tenet,7.4,"Armed with only one word, Tenet, and fighting ...","435,483",Christopher Nolan,"John David Washington, Robert Pattinson, Eliza...",69,"Action, Sci-Fi, Thriller",150 min
1,The Queen's Gambit,8.6,"Orphaned at the tender age of nine, prodigious...","364,538",NaN,"Anya Taylor-Joy, Chloe Pirrie, Bill Camp, Marc...",NaN,Drama,395 min
2,Soul,8.1,"After landing the gig of a lifetime, a New Yor...","284,827","Pete Docter, Kemp Powers","Jamie Foxx, Tina Fey, Graham Norton, Rachel House",83,"Animation, Adventure, Comedy",100 min
3,Wonder Woman 1984,5.4,Diana must contend with a work colleague and b...,"237,689",Patty Jenkins,"Gal Gadot, Chris Pine, Kristen Wiig, Pedro Pascal",60,"Action, Adventure, Fantasy",151 min
4,Birds of Prey: And the Fantabulous Emancipatio...,6.1,"After splitting with the Joker, Harley Quinn j...","214,018",Cathy Yan,"Margot Robbie, Rosie Perez, Mary Elizabeth Win...",60,"Action, Adventure, Comedy",109 min
...,...,...,...,...,...,...,...,...,...
995,Unfinished Business,8.5,The Republic plans a daring final strike again...,"3,276","Brian O'Connell, Dave Filoni","Dee Bradley Baker, Matt Lanter, Matthew Wood, ...",NaN,"Animation, Action, Adventure",23 min
996,Bad Hair,5.6,In 1989 an ambitious young woman gets a weave ...,"3,275",Justin Simien,"Zaria Kelley, Corinne Massiah, Elle Lorraine, ...",61,"Comedy, Horror, Music",102 min
997,Veneno,8.7,"A biographical story of La Veneno, transgender...","3,272",NaN,"Lola Rodríguez, Isabel Torres, Paca la Piraña,...",NaN,"Biography, Drama",421 min
998,The Undoing,7.9,Grace Fraser becomes intrigued by a young moth...,"3,271",Susanne Bier,"Nicole Kidman, Hugh Grant, Edgar Ramírez, Noah...",NaN,"Crime, Drama, Mystery",54 min


In [29]:
df.to_csv('IMDB_movies.csv')